In [62]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from skorch import NeuralNetRegressor
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error

In [63]:
 # load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt('ML-CUP23-TR.csv', delimiter=',')
X = dataset[:,1:11]
y = dataset[:,11:14]

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [73]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler

# Normalize data
#scaler = StandardScaler()
#X = scaler.fit_transform(X)
#y = scaler.fit_transform(y)

# Define a simple regression neural network
class SimpleRegressionNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRegressionNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Set hyperparameters
input_size = X.shape[1]
hidden_size = 50
output_size = 3
learning_rate = 0.001
epochs = 300

# Define the model, loss function, and optimizer
model = SimpleRegressionNN(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Define K-fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Perform K-fold cross-validation
for fold, (train_indices, val_indices) in enumerate(kf.split(X)):
    print(f"\nFold {fold + 1}/{k_folds}")

    # Split the data into training and validation sets
    X_train_tensor, X_val_tensor = X[train_indices], X[val_indices]
    y_train_tensor, y_val_tensor = y[train_indices], y[val_indices]

    # Training loop
    for epoch in range(epochs):
        # Forward pass
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 100 == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")

    # Validation
    with torch.no_grad():
        model.eval()
        val_outputs = model(X_val_tensor)
        mse = mean_squared_error(y_val_tensor.numpy(), val_outputs.numpy())
        print(f"Validation MSE: {mse:.4f}")

    model.train()  # Set the model back to training mode



Fold 1/5
Epoch 100/300, Loss: 400.3456
Epoch 200/300, Loss: 65.9189
Epoch 300/300, Loss: 15.1256
Validation MSE: 15.0391

Fold 2/5
Epoch 100/300, Loss: 10.2464
Epoch 200/300, Loss: 7.7637
Epoch 300/300, Loss: 6.2526
Validation MSE: 6.1174

Fold 3/5
Epoch 100/300, Loss: 5.1201
Epoch 200/300, Loss: 4.3383
Epoch 300/300, Loss: 3.7104
Validation MSE: 4.0962

Fold 4/5
Epoch 100/300, Loss: 3.1958
Epoch 200/300, Loss: 2.7615
Epoch 300/300, Loss: 2.4158
Validation MSE: 2.8649

Fold 5/5
Epoch 100/300, Loss: 2.2070
Epoch 200/300, Loss: 1.9753
Epoch 300/300, Loss: 1.7050
Validation MSE: 1.8050
